In [4]:
# !pip install --upgrade pip
# !pip install scikit-learn
# !pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pickle
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import openpyxl

In [2]:
# Ensure the models directory exists
os.makedirs("models", exist_ok=True)

In [ ]:
# Load and preprocess your data
def load_data(file_path):
    """Load and preprocess the work utilization data"""
    df = pd.read_excel(file_path)
    
    # Clean column names (remove whitespace)
    df.columns = df.columns.str.strip()
    
    # Ensure Date column is datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Ensure WorkType is treated as string
    df['WorkType'] = df['WorkType'].astype(str)
    
    # Fix Hours and NoOfMan columns - replace "-" with 0 and convert to numeric
    if 'Hours' in df.columns:
        df['Hours'] = df['Hours'].replace('-', 0)
        df['Hours'] = pd.to_numeric(df['Hours'], errors='coerce').fillna(0)
        
    if 'NoOfMan' in df.columns:
        df['NoOfMan'] = df['NoOfMan'].replace('-', 0)
        df['NoOfMan'] = pd.to_numeric(df['NoOfMan'], errors='coerce').fillna(0)
    
    # Sort by Date
    df = df.sort_values('Date')
    
    return df

# Feature engineering function
def engineer_features(df):
    """Create relevant features for the prediction model"""
    # Create a copy to avoid modifying the original dataframe
    data = df.copy()
    
    # Extract date features
    data['Year_feat'] = data['Date'].dt.year
    data['Month_feat'] = data['Date'].dt.month
    data['DayOfMonth'] = data['Date'].dt.day
    data['DayOfWeek_feat'] = data['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
    data['Quarter'] = data['Date'].dt.quarter
    data['IsWeekend_feat'] = data['DayOfWeek_feat'].apply(lambda x: 1 if x >= 5 else 0)
    
    # Add week of year
    data['WeekOfYear'] = data['Date'].dt.isocalendar().week
    
    # Add day of year
    data['DayOfYear'] = data['Date'].dt.dayofyear
    
    # Calculate days since the start of the dataset
    min_date = data['Date'].min()
    data['DaysSinceStart'] = (data['Date'] - min_date).dt.days
    
    return data


def create_lag_features(data, group_col='WorkType', target_col='NoOfMan', lag_days=[1, 2, 3, 7, 14, 30]):
    """Create lag features for each WorkType's NoOfMan value"""
    # Make a copy of the input dataframe
    data_copy = data.copy()
    
    # First, check if there are any non-zero values in the target column
    non_zero_count = (data_copy[target_col] > 0).sum()
    print(f"Number of non-zero {target_col} values: {non_zero_count} out of {len(data_copy)}")
    
    # # Group by WorkType and Date to get daily aggregates
    
    # Use date only if Date contains time information
    # if pd.api.types.is_datetime64_dtype(data_copy['Date']):
    #     data_copy['Date_day'] = data_copy['Date'].dt.date
    #     daily_data = data_copy.groupby([group_col, 'Date_day'])[target_col].sum().reset_index()
    #     daily_data['Date'] = pd.to_datetime(daily_data['Date_day'])
    #     daily_data = daily_data.drop('Date_day', axis=1)
    #     print('If')
    # else:
    #     daily_data = data_copy.groupby([group_col, 'Date'])[target_col].sum().reset_index()
    #     print('Else')
    
    # Add the engineered features we need
    # daily_data['DayOfWeek_feat'] = daily_data['Date'].dt.dayofweek
    # daily_data['Month_feat'] = daily_data['Date'].dt.month
    # daily_data['IsWeekend_feat'] = daily_data['DayOfWeek_feat'].apply(lambda x: 1 if x >= 5 else 0)
    # daily_data['Year_feat'] = daily_data['Date'].dt.year
    # daily_data['Quarter'] = daily_data['Date'].dt.quarter
    # daily_data['DayOfMonth'] = daily_data['Date'].dt.day
    
    # # Ensure data is properly sorted by WorkType and Date (critical for time-series operations)
    daily_data = data_copy.sort_values([group_col, 'Date'])
    
    # Create lag features for each work type
    for lag in lag_days:
        daily_data[f'{target_col}_lag_{lag}'] = daily_data.groupby(group_col)[target_col].shift(lag)
    
    # Create rolling features by work type
    for work_type in daily_data[group_col].unique():
        mask = daily_data[group_col] == work_type
        work_type_data = daily_data.loc[mask, target_col]
        
        if len(work_type_data) >= 1:
            daily_data.loc[mask, f'{target_col}_rolling_mean_7'] = work_type_data.rolling(
                window=7, min_periods=1).mean()
            daily_data.loc[mask, f'{target_col}_rolling_max_7'] = work_type_data.rolling(
                window=7, min_periods=1).max()
            daily_data.loc[mask, f'{target_col}_rolling_min_7'] = work_type_data.rolling(
                window=7, min_periods=1).min()
            daily_data.loc[mask, f'{target_col}_rolling_std_7'] = work_type_data.rolling(
                window=7, min_periods=1).std().fillna(0)
    
    # Create same day of week lag
    daily_data[f'{target_col}_same_dow_lag'] = daily_data.groupby([group_col, 'DayOfWeek_feat'])[target_col].shift(1)
    
    # Instead of dropping NaN values, fill with 0
    # This is crucial because dropping rows with NaN values might eliminate too much data
    daily_data = daily_data.fillna(0)
    return daily_data

# Build and train model for each WorkType
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import TimeSeriesSplit

def build_models(processed_data, work_types, n_splits=5):
    """Build and train a model for each WorkType using time series cross-validation"""
    models = {}
    feature_importances = {}
    metrics = {}
    
    # Define features to use
    numeric_features = [
        'NoOfMan_lag_1', 'NoOfMan_lag_2', 'NoOfMan_lag_3', 'NoOfMan_lag_7', 
        'NoOfMan_lag_14', 'NoOfMan_lag_30', 'NoOfMan_rolling_mean_7',
        'NoOfMan_rolling_max_7', 'NoOfMan_rolling_min_7', 'NoOfMan_rolling_std_7',
        'NoOfMan_same_dow_lag', 'IsWeekend_feat'
    ]
    
    categorical_features = ['DayOfWeek_feat', 'Month_feat']
    
    # Function to calculate modified MAPE with minimum threshold
    def modified_mape(y_true, y_pred, epsilon=1.0):
        """Calculate MAPE with a minimum threshold to avoid division by zero"""
        # Use max of actual value or epsilon (e.g., 1.0) as denominator
        denominator = np.maximum(np.abs(y_true), epsilon)
        return np.mean(np.abs(y_pred - y_true) / denominator) * 100
    
    for work_type in work_types:
        print(f"Building model for WorkType: {work_type}")
        
        # Filter data for this WorkType
        work_type_data = processed_data[processed_data['WorkType'] == work_type]
        
        if len(work_type_data) < 30:  # Skip if not enough data
            print(f"  Skipping {work_type}: Not enough data ({len(work_type_data)} records)")
            continue
        
        # Sort data by date to ensure time-based splitting works correctly
        work_type_data = work_type_data.sort_values('Date')
        
        # Prepare features and target
        X = work_type_data[numeric_features + categorical_features]
        y = work_type_data['NoOfMan']
        
        # Define preprocessing for categorical features
        preprocessor = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
            ],
            remainder='passthrough'
        )
        
        # Define the model pipeline
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('model', RandomForestRegressor(n_estimators=100, random_state=42))
        ])
        
        # Initialize TimeSeriesSplit with n splits
        tscv = TimeSeriesSplit(n_splits=n_splits)
        
        # Initialize metrics lists
        mae_scores = []
        rmse_scores = []
        r2_scores = []
        mape_scores = []
        
        # Perform time series cross-validation
        for train_idx, test_idx in tscv.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            # Train model
            pipeline.fit(X_train, y_train)
            
            # Make predictions
            y_pred = pipeline.predict(X_test)
            
            # Calculate metrics
            mae_scores.append(mean_absolute_error(y_test, y_pred))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
            r2_scores.append(r2_score(y_test, y_pred))
            
            # Calculate modified MAPE
            mape = modified_mape(y_test, y_pred, epsilon=1.0)
            mape_scores.append(mape)
        
        # Train final model on all data
        pipeline.fit(X, y)
        models[work_type] = pipeline
        
        # Get feature importances from the final model
        model = pipeline.named_steps['model']
        ohe = pipeline.named_steps['preprocessor'].transformers_[0][1]
        
        # Extract feature names after one-hot encoding
        cat_feature_names = []
        for i, feature in enumerate(categorical_features):
            categories = ohe.categories_[i]
            for category in categories:
                cat_feature_names.append(f"{feature}_{category}")
        
        # Combine with numeric feature names
        all_feature_names = cat_feature_names + numeric_features
        
        # Get feature importances
        importances = model.feature_importances_
        
        # Create dictionary of feature importances
        feature_importances[work_type] = dict(zip(all_feature_names, importances))
        
        # Store average metrics
        metrics[work_type] = {
            'MAE': np.mean(mae_scores),
            'RMSE': np.mean(rmse_scores),
            'R²': np.mean(r2_scores),
            'MAPE': np.mean(mape_scores)
        }
        
        print(f"  Model for {work_type} - MAE: {metrics[work_type]['MAE']:.4f}, RMSE: {metrics[work_type]['RMSE']:.4f}, R²: {metrics[work_type]['R²']:.4f}, MAPE: {metrics[work_type]['MAPE']:.2f}%")
        
        # Also print individual fold scores for detailed analysis
        print(f"  Cross-validation details:")
        for i in range(len(mae_scores)):
            print(f"    Fold {i+1}: MAE={mae_scores[i]:.4f}, RMSE={rmse_scores[i]:.4f}, R²={r2_scores[i]:.4f}, MAPE={mape_scores[i]:.2f}%")
    
    return models, feature_importances, metrics



# def build_models(processed_data, work_types):
#     """Build and train a model for each WorkType"""
#     models = {}
#     feature_importances = {}
#     metrics = {}
    
#     # Define features to use
#     numeric_features = [
#         'NoOfMan_lag_1', 'NoOfMan_lag_2', 'NoOfMan_lag_3', 'NoOfMan_lag_7', 
#         'NoOfMan_lag_14', 'NoOfMan_lag_30', 'NoOfMan_rolling_mean_7',
#         'NoOfMan_rolling_max_7', 'NoOfMan_rolling_min_7', 'NoOfMan_rolling_std_7',
#         'NoOfMan_same_dow_lag', 'IsWeekend_feat'
#     ]
    
#     categorical_features = ['DayOfWeek_feat', 'Month_feat']
    
#     for work_type in work_types:
#         print(f"Building model for WorkType: {work_type}")
        
#         # Filter data for this WorkType
#         work_type_data = processed_data[processed_data['WorkType'] == work_type]
        
#         if len(work_type_data) < 30:  # Skip if not enough data
#             print(f"  Skipping {work_type}: Not enough data ({len(work_type_data)} records)")
#             continue
            
#         # Split into train and test (use most recent 30 days as test)
#         train_data = work_type_data[work_type_data['Date'] < work_type_data['Date'].max() - timedelta(days=30)]
#         test_data = work_type_data[work_type_data['Date'] >= work_type_data['Date'].max() - timedelta(days=30)]
        
#         if len(train_data) < 30 or len(test_data) < 7:  # Skip if not enough data for training or testing
#             print(f"  Skipping {work_type}: Not enough data for train/test split")
#             continue
        
#         # Prepare features and target
#         X_train = train_data[numeric_features + categorical_features]
#         y_train = train_data['NoOfMan']
        
#         X_test = test_data[numeric_features + categorical_features]
#         y_test = test_data['NoOfMan']
        
#         # Define preprocessing for categorical features
#         preprocessor = ColumnTransformer(
#             transformers=[
#                 ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
#             ],
#             remainder='passthrough'
#         )
        
#         # Define the model pipeline
#         pipeline = Pipeline([
#             ('preprocessor', preprocessor),
#             ('model', RandomForestRegressor(n_estimators=100, random_state=42))
#         ])
        
#         # Train the model
#         pipeline.fit(X_train, y_train)
        
#         # Save the model
#         models[work_type] = pipeline
        
#         # Get feature importances
#         model = pipeline.named_steps['model']
#         ohe = pipeline.named_steps['preprocessor'].transformers_[0][1]
        
#         # Extract feature names after one-hot encoding
#         cat_feature_names = []
#         for i, feature in enumerate(categorical_features):
#             categories = ohe.categories_[i]
#             for category in categories:
#                 cat_feature_names.append(f"{feature}_{category}")
        
#         # Combine with numeric feature names
#         all_feature_names = cat_feature_names + numeric_features
        
#         # Get feature importances
#         importances = model.feature_importances_
        
#         # Create dictionary of feature importances
#         feature_importances[work_type] = dict(zip(all_feature_names, importances))
        
#         # Evaluate on test data
#         y_pred = pipeline.predict(X_test)
        
#         # Calculate metrics
#         mae = mean_absolute_error(y_test, y_pred)
#         rmse = np.sqrt(mean_squared_error(y_test, y_pred))
#         r2 = r2_score(y_test, y_pred)
        
#         # Calculate MAPE (Mean Absolute Percentage Error)
#         # Handle zero values in y_true to avoid division by zero
#         y_true_nonzero = np.array([max(0.0001, y) for y in y_test])
#         mape = np.mean(np.abs((y_true_nonzero - y_pred) / y_true_nonzero)) * 100
        
#         metrics[work_type] = {
#             'MAE': mae,
#             'RMSE': rmse,
#             'R²': r2,
#             'MAPE': mape
#         }
        
#         print(f"  Model for {work_type} - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")
    
#     return models, feature_importances, metrics


In [9]:
file_path = "C:/forlogssystems/Data/work_utilization_melted.xlsx"
df = load_data(file_path)
df.head()

,Year,Month,Week,Day,Date,DayOfweek,WorkType,Hours,NoOfMan
0,2017,1,1,2,2017-01-02,Monday,201,0.00,0.00000
24,2017,1,1,2,2017-01-02,Monday,222 Storage optimization,0.00,0.00000
23,2017,1,1,2,2017-01-02,Monday,221 Stocktaking,0.00,0.00000
22,2017,1,1,2,2017-01-02,Monday,217 Returns,0.00,0.00000
21,2017,1,1,2,2017-01-02,Monday,215 Picking Online,67.15,8.39375


In [10]:
# Show all rows and columns without truncation
import pandas as pd
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Expand display width
pd.set_option('display.max_colwidth', None)  # Show full content of cells

feature_df = engineer_features(df)
feature_df.head(10)

,Year,Month,Week,Day,Date,DayOfweek,WorkType,Hours,NoOfMan,Year_feat,Month_feat,DayOfMonth,DayOfWeek_feat,Quarter,IsWeekend_feat,WeekOfYear,DayOfYear,DaysSinceStart
0,2017,1,1,2,2017-01-02,Monday,201,0.00,0.00000,2017,1,2,0,1,0,1,2,0
24,2017,1,1,2,2017-01-02,Monday,222 Storage optimization,0.00,0.00000,2017,1,2,0,1,0,1,2,0
23,2017,1,1,2,2017-01-02,Monday,221 Stocktaking,0.00,0.00000,2017,1,2,0,1,0,1,2,0
22,2017,1,1,2,2017-01-02,Monday,217 Returns,0.00,0.00000,2017,1,2,0,1,0,1,2,0
21,2017,1,1,2,2017-01-02,Monday,215 Picking Online,67.15,8.39375,2017,1,2,0,1,0,1,2,0
20,2017,1,1,2,2017-01-02,Monday,214 Large orders,19.65,2.45625,2017,1,2,0,1,0,1,2,0
19,2017,1,1,2,2017-01-02,Monday,213 Pick3PL_Astro,34.00,4.25000,2017,1,2,0,1,0,1,2,0
17,2017,1,1,2,2017-01-02,Monday,210 Pick Sortation,48.57,6.07125,2017,1,2,0,1,0,1,2,0
16,2017,1,1,2,2017-01-02,Monday,209 Sortation,64.79,8.09875,2017,1,2,0,1,0,1,2,0
15,2017,1,1,2,2017-01-02,Monday,208 Sorter 2 SALE,0.00,0.00000,2017,1,2,0,1,0,1,2,0


In [23]:
lag_features_df = create_lag_features(feature_df)
lag_features_df.head(5)

Number of non-zero NoOfMan values: 30538 out of 73025


,Year,Month,Week,Day,Date,DayOfweek,WorkType,Hours,NoOfMan,Year_feat,Month_feat,DayOfMonth,DayOfWeek_feat,Quarter,IsWeekend_feat,WeekOfYear,DayOfYear,DaysSinceStart,NoOfMan_lag_1,NoOfMan_lag_2,NoOfMan_lag_3,NoOfMan_lag_7,NoOfMan_lag_14,NoOfMan_lag_30,NoOfMan_rolling_mean_7,NoOfMan_rolling_max_7,NoOfMan_rolling_min_7,NoOfMan_rolling_std_7,NoOfMan_same_dow_lag
0,2017,1,1,2,2017-01-02,Monday,201,0.0,0.0,2017,1,2,0,1,0,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,2017,1,1,3,2017-01-03,Tuesday,201,0.0,0.0,2017,1,3,1,1,0,1,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,2017,1,1,4,2017-01-04,Wednesday,201,0.0,0.0,2017,1,4,2,1,0,1,4,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,2017,1,1,5,2017-01-05,Thursday,201,0.0,0.0,2017,1,5,3,1,0,1,5,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,2017,1,1,6,2017-01-06,Friday,201,0.0,0.0,2017,1,6,4,1,0,1,6,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
 # Get list of unique WorkTypes
work_types = lag_features_df['WorkType'].unique()
models, feature_importances, metrics = build_models(lag_features_df, work_types, n_splits=5)

# Save models, feature importances, and metrics to files
print("Saving model files...")
with open(os.path.join("models", "work_utilization_models.pkl"), "wb") as f:
    pickle.dump(models, f)
    
with open(os.path.join("models", "work_utilization_feature_importances.pkl"), "wb") as f:
    pickle.dump(feature_importances, f)
    
with open(os.path.join("models", "work_utilization_metrics.pkl"), "wb") as f:
    pickle.dump(metrics, f)
    
print(f"Successfully saved {len(models)} models!")

Building model for WorkType: 201
  Model for 201 - MAE: 0.2459, RMSE: 0.5261, R²: 0.7098, MAPE: 15.27%
  Cross-validation details:
    Fold 1: MAE=0.1999, RMSE=0.6082, R²=0.4780, MAPE=8.68%
    Fold 2: MAE=0.4101, RMSE=0.6330, R²=0.7707, MAPE=26.54%
    Fold 3: MAE=0.1237, RMSE=0.2663, R²=0.8275, MAPE=10.35%
    Fold 4: MAE=0.1236, RMSE=0.3748, R²=0.7276, MAPE=8.62%
    Fold 5: MAE=0.3721, RMSE=0.7481, R²=0.7450, MAPE=22.17%
Building model for WorkType: 202 Goods receive G
  Model for 202 Goods receive G - MAE: 0.3158, RMSE: 0.5102, R²: 0.7643, MAPE: 18.94%
  Cross-validation details:
    Fold 1: MAE=0.2677, RMSE=0.4276, R²=0.7398, MAPE=18.56%
    Fold 2: MAE=0.2549, RMSE=0.4444, R²=0.7582, MAPE=17.22%
    Fold 3: MAE=0.2606, RMSE=0.4304, R²=0.8126, MAPE=17.02%
    Fold 4: MAE=0.3703, RMSE=0.5966, R²=0.7273, MAPE=21.49%
    Fold 5: MAE=0.4256, RMSE=0.6521, R²=0.7836, MAPE=20.41%
Building model for WorkType: 203 Goods receive D
  Model for 203 Goods receive D - MAE: 0.5939, RMSE: 1.0012